# imports

In [2]:
from useful_scit.imps2.defs import *

In [73]:
from skimage.util.shape import view_as_windows as viewW


def strided_indexing_roll(a, r):
    # Concatenate with sliced to cover all rolls
    
    assert a.shape[0]==r.shape[0], 'roll vector does not match matrix shape'
    p = np.full((a.shape[0],a.shape[1]-1),np.nan)
    a_ext = np.concatenate((p,a,p),axis=1)

    # Get sliding windows; use advanced-indexing to select appropriate ones
    n = a.shape[1]
    return viewW(a_ext,(1,n))[np.arange(len(r)), -r + (n-1),0]

# example

In [79]:
a  = np.array([
    [ 1, 2, 3],
    [ 1, 2, 3],
    [11,22,33],
    [21,22,23]
    
])

r = np.array([1,2,3,4])

In [80]:
strided_indexing_roll(a,r)

array([[nan,  1.,  2.],
       [nan, nan,  1.],
       [33., nan, nan],
       [22., 23., nan]])

In [124]:
viewW(a_ext,(1,5))[0]

array([[[nan, nan,  1.,  2.,  3.]],

       [[nan,  1.,  2.,  3., nan]],

       [[ 1.,  2.,  3., nan, nan]]])

In [65]:
a.shape[1]==r.shape[0]

False

In [94]:
%load_ext memory_profiler

# flx example

In [222]:
def f1():
    f = '/Users/diego/flexpart_management/flexpart_management/notebooks/log_pol_revisited/data/flxout_d02_20171217_000000.nc'
    d = '/Users/diego/flexpart_management/flexpart_management/notebooks/log_pol_revisited/data/header_d02.nc'

    ds = xr.open_dataset(f)
    hs = xr.open_dataset(d)

    to = hs['TOPOGRAPHY']

    shifts = np.round(to/500).astype(int)

    co = ds['CONC']
    co = co[{'Time':0,'releases':0,'ageclass':0}]


#     co.sum('south_north').plot()

    c1 = co.stack({'sw':['south_north','west_east']})
    s1 = shifts.stack({'sw':['south_north','west_east']})

    c1d = c1.values
    s1d = s1.values

    res = strided_indexing_roll(c1d.T,s1d).T

    c1d.shape,res.shape

    c2 = xr.full_like(c1,0.0) + res

    c3 = c2.unstack()

    c4 = c3.sum('south_north') - co.sum('south_north')
#     c4.plot()
    return c4 
    
    

#     c4.plot.contour()



In [240]:
# doesnt work 
def f2():
    f = '/Users/diego/flexpart_management/flexpart_management/notebooks/log_pol_revisited/data/flxout_d02_20171217_000000.nc'
    d = '/Users/diego/flexpart_management/flexpart_management/notebooks/log_pol_revisited/data/header_d02.nc'

    ds = xr.open_dataset(f)
    hs = xr.open_dataset(d)

    to = hs['TOPOGRAPHY']

    shifts = np.floor(to/500).astype(int)

    co = ds['CONC']
    co = co[{'Time':slice(0,10),'releases':0,'ageclass':0}]


#     co.sum('south_north').plot()
    sdim = ['south_north','west_east','Time']
    c1 = co.stack({'sw':sdim})
    s1 = shifts.stack({'sw':sdim})

    c1d = c1.values
    s1d = s1.values

    res = strided_indexing_roll(c1d.T,s1d).T

    c1d.shape,res.shape

    c2 = xr.full_like(c1,0.0) + res

    c3 = c2.unstack()

#     c4 = c3.sum('south_north') - co.sum('south_north')
#     c4.plot()
    return c4 

In [241]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [242]:
# %timeit c4 = f1()
%lprun -f f1 c4 = f1()
# c4.plot()


Timer unit: 1e-06 s

Total time: 1.43685 s
File: <ipython-input-222-d32d00911c61>
Function: f1 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def f1():
     2         1          2.0      2.0      0.0      f = '/Users/diego/flexpart_management/flexpart_management/notebooks/log_pol_revisited/data/flxout_d02_20171217_000000.nc'
     3         1          1.0      1.0      0.0      d = '/Users/diego/flexpart_management/flexpart_management/notebooks/log_pol_revisited/data/header_d02.nc'
     4                                           
     5         1       7237.0   7237.0      0.5      ds = xr.open_dataset(f)
     6         1      20571.0  20571.0      1.4      hs = xr.open_dataset(d)
     7                                           
     8         1        131.0    131.0      0.0      to = hs['TOPOGRAPHY']
     9                                           
    10         1       9594.0   9594.0      0.7      shifts

# from previous

In [4]:
from flexpart_alto.modules import constants as co

reload


In [7]:
def from_agl_to_asl(
        ds ,
        ds_var='conc_norm' ,
        delta_z=500 ,
        z_top=15000 ,
        ds_var_name_out=None
        ) :
    log.ger.warning(
        f'this will only work if ds z levels are constant' )
    import wrf
    
    t_list = [ co.ZM , co.R_CENTER , co.TH_CENTER ]
    d3d = ds[ ds_var ]  # .sum( [ co.RL ] )
    d3d_attrs = d3d.attrs
    d3d = d3d.transpose(
        co.RL , *t_list , transpose_coords=True
        )
    dz = d3d[ co.TOPO ] + np.round( d3d[ co.ZM ] / delta_z ) * delta_z
    d3d = d3d.reset_coords( drop=True )
    dz = dz.transpose( *t_list , transpose_coords=True )
    dz = dz.reset_coords( drop=True )
    # %%
    # print( d3d.shape )
    # print( dz.shape )
    # %%
    z_lev = np.arange( delta_z / 2 , z_top , delta_z )
    da_interp = wrf.interplevel( d3d , dz , z_lev )
    da_reinterp = da_interp.rename( level=co.ZM )

    # %%
    ds_chop = ds.isel( { co.ZM : slice( 0 , len( da_reinterp[ co.ZM ] ) ) } )
    for coord in list( ds.coords ) :
        da_reinterp = da_reinterp.assign_coords(
            **{ coord : ds_chop[ coord ] } )
    if ds_var_name_out is not None :
        da_reinterp.name = ds_var_name_out

    # we do this in order to avoid the problem of setting attributes
    # to none that cannot be saved using to netcdf.
    da_reinterp.attrs = d3d_attrs

    ds_reinterp = da_reinterp.to_dataset()
    # todo: check that concentrations are the same after resampling
    return ds_reinterp

In [8]:
f = '/Users/diego/flexpart_management/flexpart_management/notebooks/log_pol_revisited/data/flxout_d02_20171217_000000.nc'
d = '/Users/diego/flexpart_management/flexpart_management/notebooks/log_pol_revisited/data/header_d02.nc'

ds = xr.open_dataset(f)
hs = xr.open_dataset(d)

In [9]:
ds

<xarray.Dataset>
Dimensions:  (Time: 1, ageclass: 1, bottom_top: 60, releases: 24, south_north: 450, west_east: 459)
Dimensions without coordinates: Time, ageclass, bottom_top, releases, south_north, west_east
Data variables:
    Times    (Time) |S15 ...
    CONC     (Time, ageclass, releases, bottom_top, south_north, west_east) float32 ...
Attributes:
    SIMULATION_START_DATE:       20171221
    SIMULATION_START_TIME:       0
    SIMULATION_END_DATE:         20171216
    SIMULATION_END_TIME:         0
    MAP_PROJ:                    3
    EARTH_RADIUS_M:              6370000.0
    CEN_LAT:                     -17.000015
    CEN_LON:                     -66.3
    MAP_PROJ_CHAR:               Mercator
    TRUELAT1:                    30.0
    OUTLON0:                     0.0
    OUTLAT0:                     0.0
    OUTPUT_INTERVAL:             -3600
    AVERAGING_TIME:              -3600
    AVERAGE_SAMPLING:            -180
    NSPEC:                       1
    NUMRECEPTOR:                 0
    NAGECLASS:                   1
    NUMRELEASES:                 24
    DISPERSION_METHOD:           1
    SUBGRID_TOPOGRAPHY:          0
    CONVECTION_PARAM:            3
    LU_OPTION:                   1
    WEST-EAST_GRID_DIMENSION:    459
    SOUTH-NORTH_GRID_DIMENSION:  450
    BOTTOM-TOP_GRID_DIMENSION:   60
    DX:                          1055
    DY:                          1055